In [1]:
import os
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Fungsi untuk memuat gambar dari direktori
def load_images_from_directory(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        for filename in os.listdir(label_path):
            img_path = os.path.join(label_path, filename)
            images.append(img_path)
            labels.append(label)
    return images, labels

# Contoh penggunaan
dataset_path = 'D:\KULIAH\SEMESTER 5\TEKCERTAN\Proyek\CLASIFICATION SAMPAH'
X, y = load_images_from_directory(dataset_path)

# Ekstraksi fitur menggunakan CountVectorizer
# Gantilah dengan metode ekstraksi fitur yang sesuai dengan kebutuhan Anda
vectorizer = CountVectorizer()
X_features = vectorizer.fit_transform(X)

# Bagi dataset menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

# Buat model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Prediksi pada data pengujian
y_pred = rf_model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Antarmuka pengguna dengan Tkinter
class TrashClassifierApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Trash Classifier")

        self.label = tk.Label(master, text="Pilih gambar untuk diklasifikasikan:")
        self.label.pack()

        self.image_path = None
        self.image_label = tk.Label(master)
        self.image_label.pack()

        self.browse_button = tk.Button(master, text="Pilih Gambar", command=self.browse_image)
        self.browse_button.pack()

        self.classify_button = tk.Button(master, text="Klasifikasikan", command=self.classify_image)
        self.classify_button.pack()

    def browse_image(self):
        self.image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        image = Image.open(self.image_path)
        image.thumbnail((300, 300))
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

    def classify_image(self):
        if self.image_path:
            img = Image.open(self.image_path)
            img_array = vectorizer.transform([self.image_path])
            prediction = rf_model.predict(img_array)
            tk.messagebox.showinfo("Hasil Klasifikasi", f"Gambar terklasifikasi sebagai: {prediction[0]}")
        else:
            tk.messagebox.showwarning("Peringatan", "Silakan pilih gambar terlebih dahulu.")

# Membuat instance Tkinter
root = tk.Tk()
app = TrashClassifierApp(root)

# Menjalankan loop utama Tkinter
root.mainloop()


Accuracy: 100.00%


In [2]:
pip install scikit-learn pillow


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow pillow


In [10]:
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
from tensorflow.keras.preprocessing import image as img_preprocessing
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
import numpy as np

# Memuat model MobileNetV2 yang telah dilatih sebelumnya
base_model = MobileNetV2(weights='imagenet')

# Fungsi untuk memproses gambar sebelum diprediksi
def process_image(file_path):
    img = img_preprocessing.load_img(file_path, target_size=(224, 224))
    img_array = img_preprocessing.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Fungsi untuk membuat prediksi
def predict_image(file_path):
    img_array = process_image(file_path)
    predictions = base_model.predict(img_array)
    decoded_predictions = decode_predictions(predictions)
    top_prediction = decoded_predictions[0][0]
    return top_prediction[1]  # Mengembalikan label prediksi teratas

# Antarmuka pengguna dengan Tkinter
class TrashClassifierApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Pengenalan Sampah")

        self.label = tk.Label(master, text="Pilih gambar untuk diklasifikasikan:")
        self.label.pack()

        self.image_path = None
        self.image_label = tk.Label(master)
        self.image_label.pack()

        self.browse_button = tk.Button(master, text="Pilih Gambar", command=self.browse_image)
        self.browse_button.pack()

        self.classify_button = tk.Button(master, text="Klasifikasikan", command=self.classify_image)
        self.classify_button.pack()

    def browse_image(self):
        self.image_path = filedialog.askopenfilename(filetypes=[("File gambar", "*.png;*.jpg;*.jpeg")])
        image = Image.open(self.image_path)
        image.thumbnail((300, 300))
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo

    def classify_image(self):
        if self.image_path:
            try:
                predicted_class = predict_image(self.image_path)
                tk.messagebox.showinfo("Hasil Klasifikasi", f"Gambar terklasifikasi sebagai: {predicted_class}")
            except Exception as e:
                tk.messagebox.showerror("Error", f"Terjadi kesalahan: {str(e)}")
        else:
            tk.messagebox.showwarning("Peringatan", "Silakan pilih gambar terlebih dahulu.")

# Membuat instance Tkinter
root = tk.Tk()
app = TrashClassifierApp(root)

# Menjalankan loop utama Tkinter
root.mainloop()


1/1 [==============================] - 2s 2s/step


In [12]:
import os
from flask import Flask, render_template, request
from flask import send_from_directory
from werkzeug.utils import secure_filename
from PIL import Image
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

app = Flask(__name__, static_url_path='/temp')

# Set direktori dataset
dataset_dir = "CLASIFICATION SAMPAH"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")
validation_dir = os.path.join(dataset_dir, "validation")

# Buat direktori 'temp' jika belum ada
temp_dir = "temp"
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Inisialisasi classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fungsi untuk memproses dataset dan melatih model
def train_model():
    data = []
    labels = []

    # Loop melalui folder dan gambar pada setiap kategori di dalam folder 'train'
    for category in os.listdir(train_dir):
        category_path = os.path.join(train_dir, category)
        if os.path.isdir(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = Image.open(img_path).convert("RGB")
                img = img.resize((64, 64))  # Sesuaikan ukuran gambar sesuai kebutuhan

                # Konversi gambar ke array numpy
                img_array = np.array(img)
                data.append(img_array.flatten())
                labels.append(category)

    # Split data menjadi train dan test
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # Training model
    clf.fit(X_train, y_train)

    # Evaluasi model
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

# Inisialisasi model saat aplikasi dimulai
train_model()

# Fungsi untuk melakukan prediksi pada gambar yang diupload
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((64, 64))
    img_array = np.array(img).flatten()

    prediction = clf.predict([img_array])
    return prediction[0]

# Fungsi untuk menjalankan aplikasi Flask
def run_flask_app():
    @app.route("/", methods=["GET", "POST"])
    def index():
        if request.method == "POST":
            # Ambil file dari form
            uploaded_file = request.files["file"]
            if uploaded_file.filename != "":
                # Simpan file ke direktori sementara
                filename = secure_filename(uploaded_file.filename)
                temp_path = os.path.join(temp_dir, filename)
                uploaded_file.save(temp_path)

                # Lakukan prediksi
                result = predict_image(temp_path)

                # Hapus file sementara setelah digunakan
                os.remove(temp_path)

                # Tambahkan path lengkap ke direktori temp
                full_temp_path = os.path.abspath(temp_path)

                return render_template("index.html", result=result, filename=filename, full_temp_path=full_temp_path)

        return render_template("index.html", result=None, filename=None)

    @app.route('/temp/<filename>')
    def uploaded_file(filename):
        return send_from_directory(temp_dir, filename)

    app.run(debug=True)

# Jalankan aplikasi Flask di Jupyter Notebook
run_flask_app()


Accuracy: 0.9791666666666666
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\ASUS\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
